In [ ]:
import pyspark
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
import pandas as pd
import requests

# This CATALOG_URL and MANAGEMENT_URL work for the "docker compose" testing and development environment.
# Change 'server' if you are not running on "docker compose" (f. ex. 'localhost' if TIP is running locally).
CATALOG_URL = "http://server:8080/catalog"
MANAGEMENT_URL = "http://server:8080/management"

WAREHOUSE = "new_warehouse"
CATALOG = "demo_catalog"
NAMESPACE = "demo_namespace"

SPARK_VERSION = pyspark.__version__
SPARK_MINOR_VERSION = '.'.join(SPARK_VERSION.split('.')[:2])
ICEBERG_VERSION = "1.6.1"

# Create a new Catalog

In [ ]:
PROJECT_ID = "12300000-0000-0000-0000-000000000000"

response = requests.post(f"{MANAGEMENT_URL}/v1/warehouse",
              json={
                # Name of the new warehouse
                "warehouse-name": WAREHOUSE,
                # Project to create the warehouse in.
                # Projects do not need to exist before creating a warehouse in them.
                # For simple deployments that do not require multiple projects with
                # multiple warehouses each, we recommend using a single warehouse
                # with the null UUID
                "project-id": PROJECT_ID,
                # Physical location of this warehouse
                "storage-profile": {
                    "type": "s3",
                    "bucket": "examples",
                    # you can change the prefix to something else, f. ex. f"{WAREHOUSE}
                    # as long as it is unique in the bucket
                    "key-prefix": "path/to/new-warehouse/",
                    "assume-role-arn": None,
                    "endpoint": "http://minio:9000",
                    "region": "local-01",
                    "path-style-access": True,
                    "sts-enabled": False
                },
                # Storage Credentials for the profile specified above.
                # These credentials are used to grant clients access to specific files in the storage.
                # Clients do not need to know those credentials and will never obtain them directly.
                "storage-credential": {
                    "type": "s3",
                    "credential-type": "access-key",
                    "aws-access-key-id": "minio-root-user",
                    "aws-secret-access-key": "minio-root-password"
                }
            })
print(f"{response.status_code}: {response.reason}")

In [ ]:
# As warehouse names must be unique inside a project, creating the same warehouse again would fail

# Connect with Spark

In [ ]:
config = {
    f"spark.sql.catalog.{CATALOG}": "org.apache.iceberg.spark.SparkCatalog",
    f"spark.sql.catalog.{CATALOG}.type": "rest",
    f"spark.sql.catalog.{CATALOG}.uri": CATALOG_URL,
    # If you are not using the default project, prefix the warehouse name
    # with <project-id>/
    f"spark.sql.catalog.{CATALOG}.warehouse": f"{PROJECT_ID}/{WAREHOUSE}",
    f"spark.sql.catalog.{CATALOG}.io-impl": "org.apache.iceberg.aws.s3.S3FileIO",
    "spark.sql.extensions": "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
    f"spark.sql.defaultCatalog": CATALOG,
    "spark.jars.packages": f"org.apache.iceberg:iceberg-spark-runtime-{SPARK_MINOR_VERSION}_2.12:{ICEBERG_VERSION},org.apache.iceberg:iceberg-aws-bundle:{ICEBERG_VERSION}",
}

In [ ]:
spark_config = SparkConf().setMaster('local').setAppName("Iceberg-REST")
for k, v in config.items():
    spark_config = spark_config.set(k, v)

spark = SparkSession.builder.config(conf=spark_config).getOrCreate()

spark.sql(f"USE {CATALOG}")

In [ ]:
spark.sql(f"CREATE NAMESPACE IF NOT EXISTS {NAMESPACE}")
spark.sql("SHOW NAMESPACES").toPandas()

In [ ]:
data = pd.DataFrame([[1, 'a-string', 2.2]], columns=['id', 'strings', 'floats'])
sdf = spark.createDataFrame(data)

In [ ]:
sdf.writeTo(f"{NAMESPACE}.my_table").createOrReplace()

In [ ]:
spark.sql(f"SELECT * FROM {NAMESPACE}.my_table").toPandas()